In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

import os 
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
os.chdir(path)

from pyscanpath.models.CLE import CLE
from pyscanpath.models.IttiKoch import IttiKoch
from pyscanpath.models.Deepgaze import Deepgaze
from pyscanpath.models.IORROI import IORROI
from pyscanpath.utils.metrics import scaled_euclidean_distance, mannan_distance, levenshtein_distance, scaled_time_delay_embedding_distance
from pyscanpath.utils.datasets import *

/Users/matteocastagna/Documents/GitHub/castagna-pyscanpath/pyscanpath/models/Iorroi/iorroi_utils.py:185: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1708025536809/work/torch/csrc/utils/tensor_numpy.cpp:212.)
  gaussian_kernel = nn.Parameter(torch.from_numpy(gaussian_kernel).float().unsqueeze(1))


In [2]:
itti = IttiKoch()
cle = CLE()
deepgaze = Deepgaze()
iorroi = IORROI()

Using cache found in /Users/matteocastagna/.cache/torch/hub/pytorch_vision_v0.6.0
/Users/matteocastagna/anaconda3/envs/natural_interaction/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/matteocastagna/anaconda3/envs/natural_interaction/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
num_examples = 100

dataset = 'MIT1003'
data_location = 'pysaliency_datasets'

stimuli_indices = np.array(np.random.uniform(0, 1003, num_examples), dtype=np.int32)
subject_indices = np.array(np.random.uniform(0, 15, num_examples), dtype=np.int32)

stimuli_index = 100
subject_index = 0

# number of stimuli X number of metrics X number of models
metrics = np.zeros( (num_examples, 4, 4) )
# number of stimuli X scanpath length X number corrdinates X number of models
scanpaths = np.zeros( (num_examples, 10, 2, 4) )

for i in range(num_examples):
    stimuli_index = stimuli_indices[i]
    subject_index = subject_indices[i]
    img_path, img, scanpath = getDatasetScanpath(dataset, data_location, stimuli_index, subject_index)
    
    itti_scanpath = itti.getScanPath(img_path)
    cle_scanpath = cle.getScanPath(img_path)
    deepgaze_scanpath = deepgaze.getScanPath(img_path)
    iorroi_scanpath = iorroi.getScanPath(img_path)
    
    scanpath_list = [itti_scanpath, cle_scanpath, deepgaze_scanpath, iorroi_scanpath]
    scanpaths[i, :, :, 0] = itti_scanpath
    scanpaths[i, :, :, 1] = cle_scanpath
    scanpaths[i, :, :, 2] = deepgaze_scanpath
    scanpaths[i, :, :, 3] = iorroi_scanpath

    for j in range(len(scanpath_list)):
        metrics[i, 0, j] = scaled_euclidean_distance(scanpath, scanpath_list[j], img.shape[0], img.shape[1])
        metrics[i, 1, j] = mannan_distance(scanpath, scanpath_list[j], img.shape[0], img.shape[1])
        metrics[i, 2, j] = levenshtein_distance(scanpath, scanpath_list[j], img.shape[0], img.shape[1])
        metrics[i, 3, j] = scaled_time_delay_embedding_distance(scanpath, scanpath_list[j], img.shape[0], img.shape[1])
        

/Users/matteocastagna/Documents/GitHub/castagna-pyscanpath/pyscanpath/models/Deepgaze.py:48: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1708025536809/work/torch/csrc/utils/tensor_new.cpp:278.)
  image_tensor = torch.tensor([self.img.transpose(2, 0, 1)]).to(self.device)


In [ ]:
os.chdir(os.getcwd()+'/benchmarking')

indices = np.array( [stimuli_indices, subject_indices] )

np.save( 'indices', indices )
np.save('scanpaths', scanpaths)
np.save('metrics', metrics)

In [3]:
from tabulate import tabulate

metrics_load = np.load('metrics.npy')

metrics_avg = np.mean(metrics_load, 0).transpose()
metrics_std = np.std(metrics_load, 0).transpose()

headers = ['Model', 'Euclidean', 'Mannan', 'Levenshtein', 'TDE']
table = [ ['Itti-Koch'], ['CLE'], ['DeepGaze III'], ['IOR-ROI']]
for i in range(metrics_avg.shape[0]):
    row = [None] * metrics_avg.shape[1]
    for j in range(metrics_avg.shape[1]):
        row[j] = str(metrics_avg[i,j]) + u"\u00B1" + str(metrics_std[i,j])
        table[i].append(row[j])
        
print(tabulate(table, headers))

Model         Euclidean                               Mannan                                   Levenshtein                             TDE
------------  --------------------------------------  ---------------------------------------  --------------------------------------  ---------------------------------------
Itti-Koch     0.6316808246143505±0.09588030263556774  0.20325806710941155±0.15752455340286106  0.9334242424242426±0.07867148161946623  0.8239454301507567±0.02949117372864575
CLE           0.6316808246143505±0.09588030263556774  0.3205503482927721±0.21099714103625028   0.9340909090909092±0.08024111392320248  0.8239454301507567±0.02949117372864575
DeepGaze III  0.6316808246143505±0.09588030263556774  0.3530583646761098±0.19926709823016775   0.8915000000000002±0.10476150162260095  0.8239454301507567±0.02949117372864575
IOR-ROI       0.2679593789346804±0.08293430491692275  0.33358688917715024±0.22176497324534683  0.903515151515152±0.08885298057996181   0.9253581910107787±0.01694841